#### Notebooks
|  | Name | Duration* |
| ------ | ------ | ------ |
| 1. | **Introduction** | 10 + 30 |
| 2. | [Levels](https://colab.research.google.com/github/bertcoerver/pywapor_notebooks/blob/main/levels.ipynb) | 10 + 120 |
| 3. | [Composites](https://colab.research.google.com/github/bertcoerver/pywapor_notebooks/blob/main/composites.ipynb) | 10 + 10 |
| 4. | [pyWaPOR vs. WaPOR](https://colab.research.google.com/github/bertcoerver/pywapor_notebooks/blob/main/wapor_vs_pywapor.ipynb) | 10 + 30 |
| 5. | Soil Saturation | > Coming Soon < |

\* Estimation of the time required in minutes, as in "active" + "download time". 

\* Also note that running the different notebooks on the same instance (or your own computer), will save you a lot of "download time" because the downloaded data can be reused between the notebooks.

# Installation

The pywapor package depends on several other packages, most of them get installed automatically when we install pywapor. The GDAL package needs to be installed manually however. Luckily, it is already installed on the backend computer used by Google Colab. We can verify that GDAL is installed by running the following commands.

In [ ]:
from osgeo import gdal
print("Using gdal version", gdal.__version__)

Now that we know that Python is able to import the GDAL package, we can install pywapor by running the following command.



In [ ]:
!pip install pywapor==2.3.2 --quiet

If everything went well, we can now import pywapor in Python, let's try it (fingers crossed)!

In [ ]:
import pywapor
print("Using pywapor version:", pywapor.__version__)

# ETLook Input

In order to run the ETLook model, we first need to collect the necessary inputs. Two types of spatial data are required, temporal and static data. At the bottom of this notebook you can find a table specifying all the required inputs.

Each of these variables can be collected from whichever source you wish to use, as long as you make sure the units are correct, the data is stored as a GeoTIFF (1 band per file, 1 file for each variable and date), the files all have the same no-data-value and they all have the same projection and resolution.

For your convenience, the pywapor package has a function that can collect all this data from selected sources and make sure the data is stored in the correct format and folder structure. 

Because some of the data portals used require a user to login with a username and a password, we first need to set those up. Most importantly, we will need a `NASA Eearthdata Login` to be able to collect the MODIS, CHIRPS and MERRA2 datasets, which can be created [over here](https://urs.earthdata.nasa.gov/users/new).

> ⚠️
>
> After creating your account, you still need to accept some 'Terms of Use', before you can continue with this notebook. To do that, login to your newly created account and go to
>
> `Applications > Authorized Apps > Approve More Applications`
>
> There, make sure the two following applications are authorized:
> 1. `NASA GESDISC DATA ARCHIVE`
> 2. `LP DAAC Data Pool`
>
> ⚠️

Once you have your account set up, we can enter our username and password by running the following code.


In [ ]:
pywapor.collect.setup_dl_accounts.setup_account("NASA")

Next, we need a WAPOR API token. To get that token, you need to create an account [here](https://wapor.apps.fao.org/home/WAPOR_2/1) and then go to `My WaPOR > My Profile > API Token`. Once you have the token, we can add it to our pywapor accounts.

In [ ]:
pywapor.collect.setup_dl_accounts.setup_account("WAPOR")

First we define a `startdate` and `enddate`, our boundingbox through `latlim` and `lonlim` and we give a `project_folder` in which all our data will be stored. Here I've chosen an period of exactly 10 days which is the default composite length of pyWAPOR. To learn more about composites, check out the dedicated notebook here. The given boundingbox will show us the Fayoum irrigation scheme in Egypt.

Then we can run `pywapor.pre_et_look.main()` to start the downloading.

In [ ]:
project_folder = r"/Users/hmcoerver/On My Mac/pyWAPOR/example_data"
startdate = "2021-07-01"
enddate = "2021-07-11"
latlim = [28.9, 29.7]
lonlim = [30.2, 31.2]

pre_files = pywapor.pre_et_look.main(project_folder, startdate, enddate, latlim, lonlim)

After downloading the files, we can check if all the required files exist. Using the [`ls` unix command](https://en.wikipedia.org/wiki/Ls), we can see the contents of a folder, and with the asterices we can see the contents of recursive folders. Our `project_folder` now contains the following subdirectories and files.

> Actually, it also contains a folder called `RAW`, which you can see by running `!ls */*` instead. This folder holds some intermediate files that were used to create the final ETLook inputs.



In [ ]:
!ls level_1*/*

Besides the tif files, which we need as input for ETLook, a json file has also been created (named `metadata_level_1.json` here). It contains information on how the input files were created. We can check the file like this.

In [ ]:
!python -m json.tool level_1/metadata_level_1.json

Now that we know that all the required files exist, it is also a good idea to check if there is any data inside those files! For all we know right now, they might just be a bunch of empty files.

We can create a simple map using the code below.

In [ ]:
import matplotlib.pyplot as plt

tif_file = pre_files["ndvi"][0]
ndvi_data = pywapor.general.processing_functions.open_as_array(tif_file)

fig = plt.figure(1)
fig.clf()
fig.set_size_inches(4.69, 5.47)
ax = fig.gca()
pywapor.post_et_look.plot_img(ax, ndvi_data, "pywapor", "NDVI [-]")

Now that we have all the required input files its time to run the actual model!

# Running ETLook

To run the model, we need to give at least two inputs to `pywapor.et_look_code.main()`:

1.   The `project_folder`, wich we already defined.
2.   And the specific `date` for which to run the model. Here we'll use `startdate`.

In [ ]:
files = pywapor.et_look.main(project_folder, startdate)

You'll see an message indicating that `'se_root'` was `not found`. This is no problem, ETLook can calculate `se_root` with the inputs we've provided, but it can also be given as input in case you have some great soil moisture dataset lying around.

Now that the model is finished, we can check again if any new files have been created, just like we did before.

In [ ]:
!ls out_level_1*/*

If everything went well, there should be a new folder called `out_level_1`, showing several output files that have been created.



# ETLook Output

Like we did before, we can plot a map by giving a path to a tif-file.

In [ ]:
tif_file = files["et_24_mm"]
et_data = pywapor.general.processing_functions.open_as_array(tif_file)

fig = plt.figure(2)
fig.clf()
fig.set_size_inches(4.69, 5.47)
ax = fig.gca()
pywapor.post_et_look.plot_img(ax, et_data, "pywapor", "ET [mm/day]")

Or we can calculate some statistics.

In [ ]:
import numpy as np

print("resolution:", et_data.shape)
print("total pixels:", et_data.size)
print("number of pixels with missing data:", np.sum(np.isnan(et_data)))
print("maximum value: {0:.2f}".format(np.nanmax(et_data)))
print("minimum value: {0:.2f}".format(np.nanmin(et_data)))
print("mean: {0:.2f}".format(np.nanmean(et_data)))
print("median: {0:.2f}".format(np.nanmedian(et_data)))